<a href="https://colab.research.google.com/github/hamzafarooq/multi-agent-course/blob/main/Module_6/DSPy/DSPy%20Introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro to [DSPy](https://github.com/stanfordnlp/dspy?tab=readme-ov-file)

## 1. Installing the requirements

In [ ]:
!pip install dspy-ai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.3/255.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.0/256.0 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143

In [ ]:
import dspy
import sys
import os


## 2. Setting up your LM and RM

We'll start by setting up the language model (LM) and retrieval model (RM).

In this notebook, we'll work with GPT-4o and the retriever ColBERTv2.

To make things easy, we've set up a ColBERTv2 server hosting a Wikipedia 2017 "abstracts" search index (i.e., containing first paragraph of each article from this 2017 dump).


In [ ]:
turbo = dspy.LM(model = 'gpt-4o-mini')
colbertv2_wiki17_abstracts = dspy.ColBERTv2(url = 'http://20.102.90.50:2017/wiki17_abstracts')
dspy.settings.configure(lm = turbo, rm = colbertv2_wiki17_abstracts)

In [ ]:
from dspy.datasets import HotPotQA #HotPotQA dataset is used to benchmark multi-hop QA

# Load the dataset.
dataset = HotPotQA(train_seed=1, train_size=20, eval_seed=2023, dev_size=50, test_size=0) #Notice the size of training and dev dataset! Teeny tiny compared to other ML models.

# Tell DSPy that the 'question' field is the input. Any other fields are labels and/or metadata.
trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

len(trainset), len(devset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.19k [00:00<?, ?B/s]

hotpot_qa.py:   0%|          | 0.00/6.42k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/90447 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7405 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7405 [00:00<?, ? examples/s]

(20, 50)

Let's check some examples!

In [ ]:
for i in range(5):
  train_example = trainset[i]
  print(f"Question: {train_example.question}")
  print(f"Answer: {train_example.answer}", '\n')

Question: At My Window was released by which American singer-songwriter?
Answer: John Townes Van Zandt 

Question: which  American actor was Candace Kita  guest starred with 
Answer: Bill Murray 

Question: Which of these publications was most recently published, Who Put the Bomp or Self?
Answer: Self 

Question: The Victorians - Their Story In Pictures is a documentary series written by an author born in what year?
Answer: 1950 

Question: Which magazine has published articles by Scott Shaw, Tae Kwon Do Times or Southwest Art?
Answer: Tae Kwon Do Times 



As you see, not all questions are multi-hop, e.g. the very first one. But, the second question is one such question as it requires breaking up the question into pieces in order to provide an answer.

Let's check an example from the development dataset. While we will not touch this for training, we will use this for metric evaluation.

In [ ]:
dev_example = devset[18]
print(f"Question: {dev_example.question}")
print(f"Answer: {dev_example.answer}")
print(f"Relevant Wikipedia Titles: {dev_example.gold_titles}")

Question: What is the nationality of the chef and restaurateur featured in Restaurant: Impossible?
Answer: English
Relevant Wikipedia Titles: {'Robert Irvine', 'Restaurant: Impossible'}


In [ ]:
#This cell instructs how the data is presented to the model
print(f"For this dataset, training examples have input keys {train_example.inputs().keys()} and label keys {train_example.labels().keys()}")
print(f"For this dataset, dev examples have input keys {dev_example.inputs().keys()} and label keys {dev_example.labels().keys()}")

For this dataset, training examples have input keys ['question'] and label keys ['answer']
For this dataset, dev examples have input keys ['question'] and label keys ['answer', 'gold_titles']


## 3. Defining simple Signature and Predictor

In [ ]:
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = 'your api key'  #my key redacted - add your own key here

# Define the predictor.
generate_answer = dspy.Predict(BasicQA)

# Call the predictor on a particular input.
pred = generate_answer(question=dev_example.question)

# Print the input and the prediction.
print(f"Question: {dev_example.question}")
print(f"Predicted Answer: {pred.answer}")

Question: What is the nationality of the chef and restaurateur featured in Restaurant: Impossible?
Predicted Answer: American


^ If this gives an error complaining about not having and API_Key, click on the link and get a key. And run the following command:

**Wrong answer**. The chef is [Robert Irvine](https://en.wikipedia.org/wiki/Robert_Irvine), who is in fact British.


We can explore the history of this answer.

In [ ]:
turbo.inspect_history(n=1)





[2025-02-18T20:13:42.557285]

System message:

Your input fields are:
1. `question` (str)

Your output fields are:
1. `answer` (str): often between 1 and 5 words

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Answer questions with short factoid answers.


User message:

[[ ## question ## ]]
What is the nationality of the chef and restaurateur featured in Restaurant: Impossible?

Respond with the corresponding output fields, starting with the field `[[ ## answer ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`.


Response:

[[ ## answer ## ]]
American

[[ ## completed ## ]]







There is no reasoning or chain of thought in the history of the LLM provided above. Instead of using `Predict`, we will use the `ChainOfThought` module of the `DSPy`.

In [ ]:
dev_example = devset[18]
print(f"Question: {dev_example.question}")
print(f"Answer: {dev_example.answer}")
print(f"Relevant Wikipedia Titles: {dev_example.gold_titles}")


Question: What is the nationality of the chef and restaurateur featured in Restaurant: Impossible?
Answer: English
Relevant Wikipedia Titles: {'Robert Irvine', 'Restaurant: Impossible'}


In [ ]:
# Replacing the dspy.Predict(BasicQA) with dspy.ChainOfThought(BasicQA) -> Notice that the BasicQA signature in untouched.
generate_answer_with_chain_of_thought = dspy.ChainOfThought(BasicQA)

# Call the predictor on the same input.
pred = generate_answer_with_chain_of_thought(question=dev_example.question)

# Print the input, the chain of thought, and the prediction.
print(f"Question: {dev_example.question}")
print(f"Predicted Answer: {pred.answer}")

Question: What is the nationality of the chef and restaurateur featured in Restaurant: Impossible?
Predicted Answer: British


In [ ]:
turbo.inspect_history(n=1)





[2025-02-18T20:14:46.782444]

System message:

Your input fields are:
1. `question` (str)

Your output fields are:
1. `reasoning` (str)
2. `answer` (str): often between 1 and 5 words

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Answer questions with short factoid answers.


User message:

[[ ## question ## ]]
What is the nationality of the chef and restaurateur featured in Restaurant: Impossible?

Respond with the corresponding output fields, starting with the field `[[ ## reasoning ## ]]`, then `[[ ## answer ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`.


Response:

[[ ## reasoning ## ]]
The chef and restaurateur featured in "Restaurant: Impossible" is Robert Irvine, who is British. He is known for his work in the culinary indust

## 4. Retrieval and basic RAG

In [ ]:
retrieve = dspy.Retrieve(k=3)
topK_passages = None
while True:
    try:
        topK_passages = retrieve(dev_example.question).passages
        break
    except Exception as e:
        continue

print(f"Top {retrieve.k} passages for question: {dev_example.question} \n", '-' * 30, '\n')

for idx, passage in enumerate(topK_passages):
    print(f'{idx+1}]', passage, '\n')

Top 3 passages for question: What is the nationality of the chef and restaurateur featured in Restaurant: Impossible? 
 ------------------------------ 

1] Restaurant: Impossible | Restaurant: Impossible is an American reality television series, featuring chef and restaurateur Robert Irvine, that aired on Food Network from 2011 to 2016. 

2] Jean Joho | Jean Joho is a French-American chef and restaurateur. He is chef/proprietor of Everest in Chicago (founded in 1986), Paris Club Bistro & Bar and Studio Paris in Chicago, The Eiffel Tower Restaurant in Las Vegas, and Brasserie JO in Boston. 

3] List of Restaurant: Impossible episodes | This is the list of the episodes for the American cooking and reality television series "Restaurant Impossible", produced by Food Network. The premise of the series is that within two days and on a budget of $10,000, celebrity chef Robert Irvine renovates a failing American restaurant with the goal of helping to restore it to profitability and prominence.

In [ ]:
topK_passages

['Restaurant: Impossible | Restaurant: Impossible is an American reality television series, featuring chef and restaurateur Robert Irvine, that aired on Food Network from 2011 to 2016.',
 'Jean Joho | Jean Joho is a French-American chef and restaurateur. He is chef/proprietor of Everest in Chicago (founded in 1986), Paris Club Bistro & Bar and Studio Paris in Chicago, The Eiffel Tower Restaurant in Las Vegas, and Brasserie JO in Boston.',
 'List of Restaurant: Impossible episodes | This is the list of the episodes for the American cooking and reality television series "Restaurant Impossible", produced by Food Network. The premise of the series is that within two days and on a budget of $10,000, celebrity chef Robert Irvine renovates a failing American restaurant with the goal of helping to restore it to profitability and prominence. Irvine is assisted by a designer (usually Taniya Nayak, Cheryl Torrenueva, or Lynn Keagan, but sometimes Vanessa De Leon, Krista Watterworth, Yvette Irene,

In [ ]:
#check 3 passages for the same question
for i in range(3):
  while True:
    try:
      print(retrieve(dev_example.question).passages[i], '\n')
      break
    except Exception as e:
      continue

Restaurant: Impossible | Restaurant: Impossible is an American reality television series, featuring chef and restaurateur Robert Irvine, that aired on Food Network from 2011 to 2016. 

Jean Joho | Jean Joho is a French-American chef and restaurateur. He is chef/proprietor of Everest in Chicago (founded in 1986), Paris Club Bistro & Bar and Studio Paris in Chicago, The Eiffel Tower Restaurant in Las Vegas, and Brasserie JO in Boston. 

List of Restaurant: Impossible episodes | This is the list of the episodes for the American cooking and reality television series "Restaurant Impossible", produced by Food Network. The premise of the series is that within two days and on a budget of $10,000, celebrity chef Robert Irvine renovates a failing American restaurant with the goal of helping to restore it to profitability and prominence. Irvine is assisted by a designer (usually Taniya Nayak, Cheryl Torrenueva, or Lynn Keagan, but sometimes Vanessa De Leon, Krista Watterworth, Yvette Irene, or Ni

In [ ]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

In [ ]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

Time for optimizer.


In [ ]:
from dspy.teleprompt import BootstrapFewShot

# Validation logic: check that the predicted answer is correct.
# Also check that the retrieved context does actually contain that answer.
def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred) #This metric is Exact Match
    return answer_EM

# Set up a basic teleprompter, which will compile our RAG program.
teleprompter = BootstrapFewShot(metric=validate_context_and_answer) #This line bootstraps few-shot examples

# Compile!
compiled_rag = None
while True:
    try:
      compiled_rag = teleprompter.compile(RAG(), trainset=trainset)
      break
    except Exception as e:
      print(f"Exception: {str(e)}")


 30%|███       | 6/20 [00:02<00:05,  2.66it/s]

Bootstrapped 4 full traces after 6 examples for up to 1 rounds, amounting to 6 attempts.


^ It'll stop once it has reached some performance threshold.


In [ ]:
# Ask any question you like to this simple RAG program.
my_question = "What castle did David Gregory inherit?"

# Get the prediction. This contains `pred.context` and `pred.answer`.
pred = None
while True:
    try:
      pred = compiled_rag(my_question)
      break
    except Exception as e:
      continue

# Print the contexts and the answer.
print(f"Question: {my_question}")
print(f"Predicted Answer: {pred.answer}")


Question: What castle did David Gregory inherit?
Predicted Answer: Kinnairdy Castle


In [ ]:
# Let's check the retrieved passage

print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in pred.context]}")

Retrieved Contexts (truncated): ['David Gregory (physician) | David Gregory (20 December 1625 – 1720) was a Scottish physician and inventor. His surname is sometimes spelt as Gregorie, the original Scottish spelling. He inherited Kinn...', 'Gregory Tarchaneiotes | Gregory Tarchaneiotes (Greek: Γρηγόριος Ταρχανειώτης , Italian: "Gregorio Tracanioto" or "Tracamoto" ) was a "protospatharius" and the long-reigning catepan of Italy from 998 t...', 'David Gregory (mathematician) | David Gregory (originally spelt Gregorie) FRS (? 1659 – 10 October 1708) was a Scottish mathematician and astronomer. He was professor of mathematics at the University ...']


**For Readability:**

Retrieved Contexts (truncated): ['David Gregory (physician) | David Gregory (20 December 1625 – 1720) was a Scottish physician and inventor. His surname is sometimes spelt as Gregorie, the original Scottish spelling. He inherited Kinn...', 'Gregory Tarchaneiotes | Gregory Tarchaneiotes (Greek: Γρηγόριος Ταρχανειώτης , Italian: "Gregorio Tracanioto" or "Tracamoto" ) was a "protospatharius" and the long-reigning catepan of Italy from 998 t...', 'David Gregory (mathematician) | David Gregory (originally spelt Gregorie) FRS (? 1659 – 10 October 1708) was a Scottish mathematician and astronomer. He was professor of mathematics at the University ...']

**And the wikipedia link**: [David Gregory](https://en.wikipedia.org/wiki/David_Gregory_(physician)




In [ ]:
turbo.inspect_history(n=1) #To see the last context the LLM has seen. If you wanna see the previous context, you can set "n" to that number





[2025-02-18T20:17:20.708274]

System message:

Your input fields are:
1. `context` (str): may contain relevant facts
2. `question` (str)

Your output fields are:
1. `reasoning` (str)
2. `answer` (str): often between 1 and 5 words

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## context ## ]]
{context}

[[ ## question ## ]]
{question}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Answer questions with short factoid answers.


User message:

This is an example of the task, though some input or output fields are not supplied.

[[ ## context ## ]]
Not supplied for this particular example.

[[ ## question ## ]]
At My Window was released by which American singer-songwriter?

Respond with the corresponding output fields, starting with the field `[[ ## reasoning ## ]]`, then `[[ ## answer ## ]]`, and then ending with the marker for `[[

Inspect the parameters

In [ ]:
for name, parameter in compiled_rag.named_predictors():
  print(name)
  print(parameter.demos[0], '\n')


generate_answer.predict
Example({'augmented': True, 'context': ['Who Put the Bomp | Who Put The Bomp was a rock music fanzine edited and published by Greg Shaw from 1970 to 1979. Its name came from the hit 1961 doo-wop song by Barry Mann, "Who Put the Bomp". Later, the name was shortened to "Bomp!"', 'Bompiani | Bompiani is an Italian publishing house based in Milan, Italy. It was founded in 1929 by Valentino Bompiani.', "What Color is Your Parachute? | What Color is Your Parachute? by Richard Nelson Bolles is a book for job-seekers that has been in print since 1970 and has been revised every year since 1975, sometimes substantially. Bolles initially self-published the book (December 1, 1970), but it has been commercially published since November 1972 by Ten Speed Press in Berkeley, California. As of September 28, 2010, the book is available in 22 languages, it is used in 26 countries around the world, and over ten million copies have been sold worldwide. It is one of the most highly r

## 5. Evaluate the Model

In [ ]:
from dspy.evaluate import Evaluate

# Set up the `evaluate_on_hotpotqa` function.
evaluate_on_hotpotqa = Evaluate(devset=devset, num_threads=1, display_progress=True, display_table=5)

# Evaluate the `compiled_rag` program with the `answer_exact_match` metric.
metric = None
while True:
    try:
        metric = dspy.evaluate.answer_exact_match
        evaluate_on_hotpotqa(compiled_rag, metric=metric)
        break
    except Exception as e:
        continue

evaluate_on_hotpotqa(compiled_rag, metric=metric)

Average Metric: 29.00 / 50 (58.0%): 100%|██████████| 50/50 [03:20<00:00,  4.00s/it]

2025/02/18 20:20:48 INFO dspy.evaluate.evaluate: Average Metric: 29 / 50 (58.0%)


,question,example_answer,gold_titles,context,pred_answer,answer_exact_match
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{Cangzhou, Qionghai}",['Cangzhou | Cangzhou () is a prefecture-level city in eastern Heb...,No,✔️ [True]
1,Who conducts the draft in which Marc-Andre Fleury was drafted to t...,National Hockey League,"{2017–18 Pittsburgh Penguins season, 2017 NHL Expansion Draft}",['2017–18 Pittsburgh Penguins season | The 2017–18 Pittsburgh Peng...,National Hockey League,✔️ [True]
2,"The Wings entered a new era, following the retirement of which Can...",Steve Yzerman,"{Steve Yzerman, 2006–07 Detroit Red Wings season}","['Steve Yzerman | Stephen Gregory ""Steve"" Yzerman ( ; born May 9, ...",Steve Yzerman,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{Crichton Collegiate Church, Crichton Castle}","[""Crichton Collegiate Church | Crichton Collegiate Church is situa...",River Tyne,✔️ [True]
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by ...,King Alfred the Great,"{Ealhswith, Æthelweard (son of Alfred)}","[""Æthelweard of East Anglia | Æthelweard (died 854) was a 9th-cent...",King Alfred the Great,✔️ [True]


Average Metric: 29.00 / 50 (58.0%): 100%|██████████| 50/50 [00:00<00:00, 116.24it/s]

2025/02/18 20:20:49 INFO dspy.evaluate.evaluate: Average Metric: 29 / 50 (58.0%)


,question,example_answer,gold_titles,context,pred_answer,answer_exact_match
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{Cangzhou, Qionghai}",['Cangzhou | Cangzhou () is a prefecture-level city in eastern Heb...,No,✔️ [True]
1,Who conducts the draft in which Marc-Andre Fleury was drafted to t...,National Hockey League,"{2017–18 Pittsburgh Penguins season, 2017 NHL Expansion Draft}",['2017–18 Pittsburgh Penguins season | The 2017–18 Pittsburgh Peng...,National Hockey League,✔️ [True]
2,"The Wings entered a new era, following the retirement of which Can...",Steve Yzerman,"{Steve Yzerman, 2006–07 Detroit Red Wings season}","['Steve Yzerman | Stephen Gregory ""Steve"" Yzerman ( ; born May 9, ...",Steve Yzerman,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{Crichton Collegiate Church, Crichton Castle}","[""Crichton Collegiate Church | Crichton Collegiate Church is situa...",River Tyne,✔️ [True]
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by ...,King Alfred the Great,"{Ealhswith, Æthelweard (son of Alfred)}","[""Æthelweard of East Anglia | Æthelweard (died 854) was a 9th-cent...",King Alfred the Great,✔️ [True]


58.0

In [ ]:
def gold_passages_retrieved(example, pred, trace=None):
    gold_titles = set(map(dspy.evaluate.normalize_text, example['gold_titles']))
    found_titles = set(map(dspy.evaluate.normalize_text, [c.split(' | ')[0] for c in pred.context]))

    return gold_titles.issubset(found_titles)
compiled_rag_retrieval_score = None

while True:
    try:
        compiled_rag_retrieval_score = evaluate_on_hotpotqa(compiled_rag, metric=gold_passages_retrieved)
        break
    except Exception as e:
        continue

Average Metric: 13.00 / 50 (26.0%): 100%|██████████| 50/50 [00:00<00:00, 109.42it/s]

2025/02/18 20:20:49 INFO dspy.evaluate.evaluate: Average Metric: 13 / 50 (26.0%)


,question,example_answer,gold_titles,context,pred_answer,gold_passages_retrieved
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{Cangzhou, Qionghai}",['Cangzhou | Cangzhou () is a prefecture-level city in eastern Heb...,No,
1,Who conducts the draft in which Marc-Andre Fleury was drafted to t...,National Hockey League,"{2017–18 Pittsburgh Penguins season, 2017 NHL Expansion Draft}",['2017–18 Pittsburgh Penguins season | The 2017–18 Pittsburgh Peng...,National Hockey League,✔️ [True]
2,"The Wings entered a new era, following the retirement of which Can...",Steve Yzerman,"{Steve Yzerman, 2006–07 Detroit Red Wings season}","['Steve Yzerman | Stephen Gregory ""Steve"" Yzerman ( ; born May 9, ...",Steve Yzerman,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{Crichton Collegiate Church, Crichton Castle}","[""Crichton Collegiate Church | Crichton Collegiate Church is situa...",River Tyne,✔️ [True]
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by ...,King Alfred the Great,"{Ealhswith, Æthelweard (son of Alfred)}","[""Æthelweard of East Anglia | Æthelweard (died 854) was a 9th-cent...",King Alfred the Great,


For more advanced topics, refer to https://github.com/stanfordnlp/dspy/tree/main


<h3 align="center"></h3>


<h3 align="center">---Son---</h3>








